In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import validation_curve
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, ConfusionMatrixDisplay, precision_score, \
    recall_score, roc_auc_score, roc_curve
import joblib

In [2]:
processed_train_data=pd.read_csv('../data/processed data/processed_train_data.csv', encoding='ISO-8859-1')
processed_test_data=pd.read_csv('../data/processed data/processed_val_data.csv', encoding='ISO-8859-1')

In [ ]:
X_train=processed_train_data.drop('target', axis=1)
y_train=processed_train_data['target']
X_test=processed_test_data.drop('target',axis=1)
y_test=processed_test_data['target']

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train.head(), y_train.head()

In [3]:
processed_df.head()

,target,text
0,0,api call
1,0,okay may sound silly good use oil sound awesom...
2,0,lunch not food go
3,0,suppose communicate twitter ahhh hard miss sc...
4,1,goodmorning everyone go degree outside today ...


In [3]:
processed_df.dropna(inplace=True)
processed_df.reset_index(drop=True, inplace=True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    processed_df,
    processed_df['target'],
    test_size=0.3, random_state=1234, stratify=processed_df['target']
)

In [6]:
X_train.head(), X_test.head()

(        target                                               text
 683051       0                          see alot pleat khaki pant
 229135       0                   wish new season hero stop season
 108440       0  good nite tweet twitter time nyquil get rid bu...
 20850        1                 everything come back david tennant
 668462       1                                   4hours boom boom,
         target                                               text
 663294       0                       thats terrible  glad catch b
 854          1  wel busy entire day let c come onlinetill priv...
 253398       0  remember tried dance team not make  tina could...
 551920       1          saw wee singsong last night log  nice one
 562243       0             vicar dibley always make cry  love bit)

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506194 entries, 683051 to 284351
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   target  506194 non-null  int64 
 1   text    506194 non-null  object
dtypes: int64(1), object(1)
memory usage: 11.6+ MB


## TF-IDF

In [5]:
tfidf = TfidfVectorizer(norm='l2')
tfidf_vector_train=tfidf.fit_transform(X_train['text'])
tfidf_vector_test = tfidf.transform(X_test['text'])

tfidf_biagrams = TfidfVectorizer(ngram_range=(1, 2),norm='l2')
tfidf_biagrams_vector_train=tfidf_biagrams.fit_transform(X_train['text'])
tfidf_biagram_vector_test = tfidf_biagrams.transform(X_test['text'])


tfidf_trigrams = TfidfVectorizer(ngram_range=(1, 3),norm='l2')
tfidf_trigrams_vector_train=tfidf_trigrams.fit_transform(X_train['text'])
tfidf_trigrams_vector_test = tfidf_trigrams.transform(X_test['text'])

In [14]:
print(tfidf_vector_train)

  (0, 119054)	0.38249411204964373
  (0, 88966)	0.5642478788341421
  (0, 123334)	0.6034713717865785
  (0, 16543)	0.35937130887716734
  (0, 137947)	0.20493206500804764
  (1, 148612)	0.323851344987834
  (1, 75499)	0.44875203603472036
  (1, 137752)	0.7490271823985599
  (1, 109985)	0.26347880137926866
  (1, 174321)	0.2515534093509417
  (2, 112429)	0.10174966441685704
  (2, 26748)	0.3322139483148243
  (2, 176148)	0.18365739052734814
  (2, 173157)	0.39201568733928377
  (2, 140664)	0.28313458682786735
  (2, 32513)	0.28882351439392234
  (2, 132611)	0.2960260741901003
  (2, 67348)	0.12140530664499194
  (2, 113718)	0.3948740854678386
  (2, 157279)	0.16101709514276127
  (2, 163093)	0.18625530457712983
  (2, 162333)	0.20695181323252412
  (2, 111200)	0.27434658547369734
  (2, 69070)	0.30212823695744623
  (3, 154235)	0.6805990734323293
  :	:
  (506189, 103656)	0.10120113666086965
  (506190, 20691)	0.45294601952772895
  (506190, 74303)	0.38720077056751334
  (506190, 145307)	0.40773460407299994
  (5061

# Basic Models

In [22]:
models = {
    'LGBMClassifier': LGBMClassifier(),
    'XGBClassifier': XGBClassifier(objective='binary:logistic', 
                                   eval_metric='logloss', 
                                   n_estimators=100, 
                                   max_depth=5),
     'LinearSVC': LinearSVC(),
     'LogisticRegression': LogisticRegression(),
     'MultinomialNB': MultinomialNB(),
     'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=10,
                                                       min_samples_split=10,
                                                       min_samples_leaf=5,
                                                       max_features='sqrt',
                                                       criterion='gini')
}

classification_reports = {}

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(tfidf_trigrams_vector_train,y_train)

    y_train_pred=model.predict(tfidf_trigrams_vector_train)
    y_test_pred=model.predict(tfidf_trigrams_vector_test)

    report = classification_report(y_test, y_test_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()

    classification_reports[list(models.keys())[i]] = report_df

    #Training set performence
    model_train_accuracy=accuracy_score(y_train,y_train_pred)
    model_train_f1=f1_score(y_train,y_train_pred, average='weighted')
    model_train_precision=precision_score(y_train,y_train_pred)
    model_train_recall=recall_score(y_train,y_train_pred)
    model_train_roc_auc_score=roc_auc_score(y_train,y_train_pred)

    #Test set performence
    model_test_accuracy=accuracy_score(y_test,y_test_pred)
    model_test_f1=f1_score(y_test,y_test_pred, average='weighted')
    model_test_precision=precision_score(y_test,y_test_pred)
    model_test_recall=recall_score(y_test,y_test_pred)
    model_test_roc_auc_score=roc_auc_score(y_test,y_test_pred)

    print(list(models.keys())[i].upper())
    print('==================================')

    print('Model Performence for Training set:')
    print('- F1 Score: {:.4f}'.format(model_train_f1))
    print('- Precision: {:.4f}'.format(model_train_precision))
    print('- Recall: {:.4f}'.format(model_train_recall))
    print('- ROC AUC Score: {:.4f}'.format(model_train_roc_auc_score))

    print('----------------------------------')
    print('Model Performence for Test set:')
    print('- Accuracy: {:.4f}'.format(model_test_accuracy))
    print('- F1 Score: {:.4f}'.format(model_test_f1))
    print('- Precision: {:.4f}'.format(model_test_precision))
    print('- Recall: {:.4f}'.format(model_test_recall))
    print('- ROC AUC Score: {:.4f}'.format(model_test_roc_auc_score))
    print('----------------------------------')
    print('Report Test set:')
    print(report_df)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{list(models.keys())[i]}_{timestamp}.pkl"
    joblib.dump(model, filename)
    print(f"\nModel saved as {filename}")
    print('==================================')

    print('\n')


[LightGBM] [Info] Number of positive: 119746, number of negative: 386375
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 81.896936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 423390
[LightGBM] [Info] Number of data points in the train set: 506121, number of used features: 29550
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.236596 -> initscore=-1.171436
[LightGBM] [Info] Start training from score -1.171436
LGBMCLASSIFIER
Model Performence for Training set:
- F1 Score: 0.7912
- Precision: 0.7637
- Recall: 0.3393
- ROC AUC Score: 0.6534
----------------------------------
Model Performence for Test set:
- Accuracy: 0.8151
- F1 Score: 0.7925
- Precision: 0.7095
- Recall: 0.3704
- ROC AUC Score: 0.6617
----------------------------------
Report Test set:
              precision    recall  f1-score        support
0              0.830042  

c:\Users\abc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LOGISTICREGRESSION
Model Performence for Training set:
- F1 Score: 0.8789
- Precision: 0.9212
- Recall: 0.5786
- ROC AUC Score: 0.7816
----------------------------------
Model Performence for Test set:
- Accuracy: 0.8433
- F1 Score: 0.8329
- Precision: 0.7417
- Recall: 0.5183
- ROC AUC Score: 0.7312
----------------------------------
Report Test set:
              precision    recall  f1-score        support
0              0.863445  0.944054  0.901952  165589.000000
1              0.741669  0.518258  0.610156   51320.000000
accuracy       0.843312  0.843312  0.843312       0.843312
macro avg      0.802557  0.731156  0.756054  216909.000000
weighted avg   0.834633  0.843312  0.832914  216909.000000

Model saved as LogisticRegression_20240406_143042.pkl


MULTINOMIALNB
Model Performence for Training set:
- F1 Score: 0.7298
- Precision: 0.9942
- Recall: 0.1360
- ROC AUC Score: 0.5679
----------------------------------
Model Performence for Test set:
- Accuracy: 0.7744
- F1 Score: 0.6867
-

## Linear SVC for TF-IDF

In [15]:
l_SVC=LinearSVC()
l_SVC.fit(tfidf_vector_train,y_train)
print(f"Train Accuracy: {l_SVC.score(tfidf_vector_train, y_train) * 100:.3f}%", )
print(f"Test Accuracy: {l_SVC.score(tfidf_vector_test, y_test) * 100:.3f}%")

Train Accuracy: 89.720%
Test Accuracy: 83.143%


In [16]:
y_pred_test = l_SVC.predict(tfidf_vector_test)
report = classification_report(y_test, y_pred_test, output_dict=True)

report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
0,0.858586,0.932825,0.894167,165613.000000
1,0.699389,0.504267,0.586012,51328.000000
accuracy,0.831429,0.831429,0.831429,0.831429
macro avg,0.778988,0.718546,0.740090,216941.000000
weighted avg,0.820920,0.831429,0.821258,216941.000000


### Using biagrams

In [17]:
l_SVC_biagrams=LinearSVC()
l_SVC_biagrams.fit(tfidf_biagrams_vector_train,y_train)
print(f"Train Accuracy: {l_SVC_biagrams.score(tfidf_biagrams_vector_train, y_train) * 100:.3f}%", )
print(f"Test Accuracy: {l_SVC_biagrams.score(tfidf_biagram_vector_test, y_test) * 100:.3f}%")

Train Accuracy: 98.558%
Test Accuracy: 84.123%


In [18]:
y_pred_biagram_test = l_SVC_biagrams.predict(tfidf_biagram_vector_test)
report = classification_report(y_test, y_pred_biagram_test, output_dict=True)

report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
0,0.870369,0.930633,0.899493,165613.000000
1,0.711798,0.552778,0.622290,51328.000000
accuracy,0.841233,0.841233,0.841233,0.841233
macro avg,0.791084,0.741706,0.760892,216941.000000
weighted avg,0.832852,0.841233,0.833907,216941.000000


### Using trigrams

In [6]:
l_SVC_trigrams=LinearSVC(class_weight='balanced')
l_SVC_trigrams.fit(tfidf_trigrams_vector_train,y_train)
print(f"Train Accuracy: {l_SVC_trigrams.score(tfidf_trigrams_vector_train, y_train) * 100:.3f}%", )
print(f"Test Accuracy: {l_SVC_trigrams.score(tfidf_trigrams_vector_test, y_test) * 100:.3f}%")

Train Accuracy: 99.298%
Test Accuracy: 83.166%


In [7]:
y_pred_trigrams_test = l_SVC_trigrams.predict(tfidf_trigrams_vector_test)
report = classification_report(y_test, y_pred_trigrams_test, output_dict=True)

report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
0,0.894151,0.884152,0.889123,165613.000000
1,0.639225,0.662290,0.650553,51328.000000
accuracy,0.831659,0.831659,0.831659,0.831659
macro avg,0.766688,0.773221,0.769838,216941.000000
weighted avg,0.833836,0.831659,0.832678,216941.000000


### Hyperparameters for LinearSVC
using trigrams tfidf vectors

In [8]:
#RandomGridSearch
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, make_scorer

param_grid = {
    'C': reciprocal(0.001, 1000),   # Rozkład odwrotnie proporcjonalny
    'penalty': ['l1', 'l2'],        # Rodzaj regularyzacji
    'dual': [False],                # Ustawienie na False dla dużej liczby próbek
    'tol': [1e-3, 1e-4, 1e-5],      # Tolerancja dla kryterium stopu
    'max_iter': [1000, 2000, 3000]  # Maksymalna liczba iteracji
}

small_tfidf_trigrams_vector_train = tfidf_trigrams_vector_train[:50000]
small_y_train = y_train[:50000]

roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True)

random = RandomizedSearchCV(estimator=l_SVC_trigrams, param_distributions=param_grid,
                            cv=3, n_jobs=-1, n_iter=10, scoring=roc_auc_scorer)

with tqdm(total=10, desc="Random Grid Search") as pbar:
    random_result = random.fit(small_tfidf_trigrams_vector_train, small_y_train)
    pbar.update(1)

print("Best ROC AUC score: %f using %s" % (random_result.best_score_, random_result.best_params_))

Random Grid Search:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
#dla 1000
#cv=3 Best ROC AUC score: 0.581979 using {'C': 1.6710219312468042, 'dual': False, 'max_iter': 2000, 'penalty': 'l1', 'tol': 0.001}
#cv=6 Best ROC AUC score: 0.592834 using {'C': 1.8006074553463385, 'dual': False, 'max_iter': 1000, 'penalty': 'l1', 'tol': 1e-05}

In [ ]:
#dla 5000
#cv=3 Best ROC AUC score: 0.680294 using {'C': 0.41490422570295576, 'dual': False, 'max_iter': 3000, 'penalty': 'l1', 'tol': 0.0001}

In [ ]:
#dla 50 000
#cv=3 

In [ ]:
#Best ROC AUC score: 0.598597 using {'C': 180.05276095382368, 'dual': False, 'max_iter': 2000, 'penalty': 'l1', 'tol': 0.001}
#Best ROC AUC score: 0.663885 using {'C': 1.9191191218692307, 'dual': False, 'max_iter': 3000, 'penalty': 'l2', 'tol': 1e-05}

In [ ]:
best_model=random_result.best_estimator_
best_model.score(X_test, y_test)

In [6]:
# Not using gamma and kernel since we use LinearSVC instead of SVC.

param_grid = {'C': [0.1, 1, 10]}

linearSVC_grid = GridSearchCV(LinearSVC(), param_grid, refit=True, verbose=3)
linearSVC_grid.fit(tfidf_trigrams_vector_train, y_train)

print("Tuned hyperparameters (best parameters): ", linearSVC_grid.best_params_)
print("Accuracy:", linearSVC_grid.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[CV 1/5] END .............................C=0.1;, score=0.835 total time=   6.6s
[CV 2/5] END .............................C=0.1;, score=0.832 total time=   4.1s
[CV 3/5] END .............................C=0.1;, score=0.833 total time=   4.1s
[CV 4/5] END .............................C=0.1;, score=0.833 total time=   4.1s
[CV 5/5] END .............................C=0.1;, score=0.832 total time=   4.2s
[CV 1/5] END ...............................C=1;, score=0.841 total time=  12.8s
[CV 2/5] END ...............................C=1;, score=0.840 total time=  12.5s
[CV 3/5] END ...............................C=1;, score=0.839 total time=  12.2s
[CV 4/5] END ...............................C=1;, score=0.840 total time=  12.3s
[CV 5/5] END ...............................C=1;, score=0.840 total time=  12.7s
[CV 1/5] END ..............................C=10;, score=0.828 total time= 3.0min
[CV 2/5] END ..............................C=10;, score=0.829 total time= 3.2min
[CV 3/5] END ...............

## Logistic Regression

In [10]:
lr=LogisticRegression(solver='sag')
lr.fit(tfidf_vector_train,y_train)
print(f"Train Accuracy: {lr.score(tfidf_vector_train, y_train) * 100:.3f}%", )
print(f"Test Accuracy: {lr.score(tfidf_vector_test, y_test) * 100:.3f}%")

Train Accuracy: 85.790%
Test Accuracy: 83.763%


In [11]:
y_pred_train = lr.predict(tfidf_vector_test)
report = classification_report(y_test, y_pred_train, output_dict=True)

report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
0,0.856170,0.946271,0.898968,165589.000000
1,0.737505,0.487081,0.586688,51320.000000
accuracy,0.837628,0.837628,0.837628,0.837628
macro avg,0.796838,0.716676,0.742828,216909.000000
weighted avg,0.828094,0.837628,0.825084,216909.000000


### Hyperparameters for logistic regression

In [6]:
grid = {"C": np.logspace(-3, 3, 7), "penalty": ["l1", "l2"], "solver": ["liblinear", "saga"]} # l1 lasso l2 ridge
logreg=LogisticRegression(max_iter=100)
logreg_cv=GridSearchCV(logreg,grid,cv=3,verbose=3,refit=True)
logreg_cv.fit(tfidf_vector_train,y_train)

print("tuned hpyerparameters :(best parameters) ", logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV 1/3] END C=0.001, penalty=l1, solver=liblinear;, score=0.763 total time=   2.0s
[CV 2/3] END C=0.001, penalty=l1, solver=liblinear;, score=0.763 total time=   0.9s
[CV 3/3] END C=0.001, penalty=l1, solver=liblinear;, score=0.763 total time=   1.1s
[CV 1/3] END ..C=0.001, penalty=l1, solver=saga;, score=0.763 total time=   6.9s
[CV 2/3] END ..C=0.001, penalty=l1, solver=saga;, score=0.763 total time=   7.0s
[CV 3/3] END ..C=0.001, penalty=l1, solver=saga;, score=0.763 total time=   5.2s
[CV 1/3] END C=0.001, penalty=l2, solver=liblinear;, score=0.763 total time=   0.7s
[CV 2/3] END C=0.001, penalty=l2, solver=liblinear;, score=0.763 total time=   0.8s
[CV 3/3] END C=0.001, penalty=l2, solver=liblinear;, score=0.763 total time=   0.8s
[CV 1/3] END ..C=0.001, penalty=l2, solver=saga;, score=0.763 total time=  14.2s
[CV 2/3] END ..C=0.001, penalty=l2, solver=saga;, score=0.763 total time=  12.9s
[CV 3/3] END ..C=0.001, penalt

## CNN Network - Convolutional Neural Networks